In [1]:
import numpy as np
import pandas as pd
import matplotlib.backends.backend_pdf
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from itertools import combinations


import scanpy as sc 
import anndata as ad
import tangram as tg
import matplotlib.pyplot as plt
import seaborn as sns
import copy

np.random.seed(0)
from tqdm.notebook import tqdm



In [2]:
project_location = "./"


In [4]:
# Load data

In [3]:
df_st = pd.read_csv(project_location + "data/merfish/merfishSpatial.csv", sep=" ")
coords = df_st['coord']
df_st.drop(columns=['coord'], inplace=True)
cellTypeCount =16
typeCountDf=  df_st.iloc[:,-cellTypeCount:]
# df_st = df_st.iloc[: , :-cellTypeCount]
df_st.head()

,Ace2,Adora2a,Aldh1l1,Amigo2,Ano3,Aqp4,Ar,Arhgap36,Avpr1a,Avpr2,...,ODImmature1,ODImmature2,Microglia,ODMature2,ODMature1,Endothelial3,ODMature3,ODMature4,Endothelial2,Ependymal
0,3.993327,20.596571,7.531108,13.294050,5.552498,9.445471,9.026888,23.262853,1.559171,0.000000,...,1,0,0,1,2,1,0,0,0,0
1,0.000000,0.000000,10.676513,0.606719,0.000000,43.836144,0.000000,8.300263,0.606719,1.437811,...,0,0,0,4,0,0,0,0,0,0
2,0.000000,0.000000,3.378244,1.219116,1.219116,7.910641,0.000000,1.529895,1.102756,0.000000,...,0,0,0,2,1,0,0,0,0,0
3,0.000000,0.000000,1.465245,0.732607,0.000000,2.323503,0.795436,0.732607,0.732607,0.000000,...,0,0,0,2,1,0,0,0,0,0
4,6.311896,7.574326,0.600405,0.600405,0.600405,0.600405,0.000000,0.600405,0.000000,0.000000,...,0,0,1,1,0,1,0,0,0,0


In [4]:
reference = pd.read_csv(project_location + "data/merfish/markerGene_for_merfish_data.csv")

reference.drop(columns=['p_value'], inplace=True)
# reference.drop(range(148,168))
reference.drop(reference.index[reference.cell_type=='EpendymalInhibitory'].tolist(), inplace=True)
reference['cell_type'] = reference['cell_type'].astype(str).str.replace(" ", "")
reference.head()

,cell_type,marker_gene
0,Astrocyte,Aldh1l1
1,Astrocyte,Aqp4
2,Astrocyte,Cxcl14
3,Astrocyte,Mlc1
4,Astrocyte,Ttyh2


In [5]:
genes = df_st.columns[:155]
markers = reference.groupby('cell_type').agg(list).marker_gene
celltypes = reference.cell_type.unique().tolist()
expressions = df_st.drop(columns=celltypes)
expressions = MinMaxScaler().fit_transform(expressions)
counts_st = pd.DataFrame(expressions, columns=genes)
cellcount = pd.DataFrame(np.sum(df_st[celltypes], axis=1), columns=['cellcount'])

## Single-cell data

In [6]:
df_sc = pd.read_csv(project_location + "data/merfish/merfishVisium.csv")
df_sc.head()

,Cell_ID,Animal_ID,Animal_sex,Behavior,Bregma,Centroid_X,Centroid_Y,Cell_class,Neuron_cluster_ID,Ace2,...,Penk,Scg2,Sln,Sst,Tac1,Tac2,Th,Trh,Ucn3,Vgf
0,1f9a8c19-b089-43d1-b609-7e791dc2c70f,1,Female,Naive,-0.24,-3749.176078,-3749.458442,Astrocyte,NaN,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.273327,0.0,0.003225,0.000000
1,b13e98f4-5c2b-4e96-985e-3e93aedc7221,1,Female,Naive,-0.24,-3746.226210,-3742.259347,Inhibitory,I-1,0.0,...,0.097542,0.0,0.000000,0.0,0.000000,0.016809,0.293094,0.0,0.006602,0.018258
2,d06cb29e-10ee-4bbc-b74e-90237999ef4b,1,Female,Naive,-0.24,-3742.897643,-3790.648737,Inhibitory,I-13,0.0,...,0.087302,0.0,0.063460,0.0,0.072791,0.000000,0.289555,0.0,0.000000,0.018325
3,12e2a165-57c7-4f37-96dd-23f6574af4ba,1,Female,Naive,-0.24,-3738.423005,-3773.259265,Inhibitory,I-19,0.0,...,0.098912,0.0,0.000000,0.0,0.048410,0.000000,0.247843,0.0,0.000000,0.008280
4,14a0f396-b13d-4d45-89a3-86c2047bf3f9,1,Female,Naive,-0.24,-3736.407620,-3895.590306,Pericytes,NaN,0.0,...,0.047054,0.0,0.178222,0.0,0.000000,0.000000,0.294404,0.0,0.000000,0.000000


In [7]:
counts_sc = df_sc[genes]

annotation= df_sc["Cell_class"]
frequency={}
for item in annotation:
    if item in frequency:
        frequency[item]+=1
    else:
        frequency[item]=1

frequency


{'Astrocyte': 745,
 'Inhibitory': 2219,
 'Pericytes': 62,
 'Ambiguous': 869,
 'Endothelial1': 301,
 'Excitatory': 1219,
 'ODImmature1': 223,
 'ODImmature2': 7,
 'Microglia': 106,
 'ODMature2': 241,
 'ODMature1': 49,
 'Endothelial3': 117,
 'ODMature3': 6,
 'ODMature4': 10,
 'Endothelial2': 19,
 'Ependymal': 219}

### function for picking top five single cell of  two types

In [7]:
def  singleCellDf(twoTypes):
    global genes
    
    firstDf= df_sc.loc[df_sc['Cell_class']==twoTypes[0]]
    secondDf=df_sc.loc[df_sc['Cell_class']==twoTypes[1]]
    firstDf= firstDf.head(5)
    secondDf =secondDf.head(5)
    resultDf =pd.concat([firstDf,secondDf])
    fileName = "./data/twoType/topFive/"+twoTypes[0]+ twoTypes[1]+"Single.csv"
    resultDf.to_csv(fileName,index=False)
    # resultDf= pd.DataFrame(np.repeat(resultDf.to_numpy(), 126, axis=0), columns=firstDf.columns)
    # fileName = "./data/twoType/singleCell/"+twoTypes[0]+ twoTypes[1]+"Single.csv"
    # resultDf.to_csv(fileName,index= False)
    print(resultDf.shape)


    firstDf = firstDf[genes]
    secondDf= secondDf[genes]

    return [firstDf,secondDf]
   
    
   

### functon for picking all single cell of particular two types  

In [8]:
def  allSingleCellTwoTypes(twoTypes):
    global genes
    
    firstDf= df_sc.loc[df_sc['Cell_class']==twoTypes[0]]
    secondDf=df_sc.loc[df_sc['Cell_class']==twoTypes[1]]
    resultDf =pd.concat([firstDf,secondDf])
    types = resultDf['Cell_class']
    resultDf= resultDf[genes]
    resultDf["type"] = types
    fileName = "./data/twoType/allSingleCell/"+twoTypes[0]+ twoTypes[1]+"Single.csv"
    resultDf.to_csv(fileName,index=False)
    print(resultDf.shape)


    

    return [firstDf,secondDf]

getting permutation of equal cell counts

In [13]:
def permutationOnaspot(firstDf,secondDf,twoType):
    arr=[0,1,2,3,4]
    global genes
    
    name1= twoType[0]+"cells"
    name2= twoType[1]+ "cells"

    
    combData=  pd.DataFrame(columns=genes)
    permTracker=[]
    permFirst=[]
    permSecond=[]

    for i in range(1,6,1):
            

        permList =list(combinations(arr,i))
        permList = [list(elem) for elem in permList]
        
        tempData=  pd.DataFrame(columns=genes)

        for m in permList:
            for n in permList:
                exRows=firstDf.iloc[m,:]
                inRows= secondDf.iloc[n,:]
                exRows = pd.concat([exRows,inRows])
            
                spot= exRows.sum(axis=0)  
                
                tempData = tempData.append(spot,ignore_index=True)
                permTracker.append([m,n])
                permFirst.append(m)
                permSecond.append(n)
                
                
        tempData[twoType[0]]= i
        tempData[twoType[1]] =i
        combData=pd.concat([combData,tempData])


    stringVal=[]
    for i in range(len(combData)):
        res= str(i)+"x"+ str(i)
        stringVal.append(res)
    combData.insert(0,"coord",stringVal)
    combData["coord"]=combData.coord.astype(str)

    # combData[str(twoType[0])+"cells"]=permFirst
    # combData[str(twoType[1])+"cells"]=permSecond



    combData.to_csv("./data/twoType/spatialData/"+ twoType[0]+twoType[1]+ "Spatial.csv",index= False)



In [14]:
# dataCellTypes= ["Astrocyte","Inhibitory","Pericytes","Ambiguous","Endothelial1","Excitatory","ODImmature1","ODImmature2","Microglia","ODMature2",
# "ODMature1","Endothelial3","Ependymal","Endothelial2","ODMature3","ODMature4"]
dataCellTypes= ["Inhibitory","Excitatory"]
twoTypeList =list(combinations(dataCellTypes,2))
twoTypeList = [list(elem) for elem in twoTypeList]
for t in twoTypeList:
    firstTypeDf,secondTypeDf= singleCellDf(t)
    permutationOnaspot(firstTypeDf,secondTypeDf,t)
    


(30, 164)
